task name: Resolve NormalizeLogMeanVariance Task
task description: Implement the NormalizeLogMeanVariance example using a dotnet interactive notebook.

In [ ]:
#r "nuget:Microsoft.ML"


In [ ]:
using System;
using System.Collections.Generic;
using System.Collections.Immutable;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;
using static Microsoft.ML.Transforms.NormalizingTransformer;


In [ ]:
private class DataPoint
{
    [VectorType(5)]
    public float[] Features { get; set; }
}

In [ ]:
var mlContext = new MLContext();
var samples = new List<DataPoint>()
{
    new DataPoint(){ Features = new float[5] { 1, 1, 3, 0, float.MaxValue } },
    new DataPoint(){ Features = new float[5] { 2, 2, 2, 0, float.MinValue } },
    new DataPoint(){ Features = new float[5] { 0, 0, 1, 0, 0} },
    new DataPoint(){ Features = new float[5] {-1,-1,-1, 1, 1} }
};

In [ ]:
var data = mlContext.Data.LoadFromEnumerable(samples);

In [ ]:
var normalize = mlContext.Transforms.NormalizeLogMeanVariance("Features", useCdf: true);
var normalizeNoCdf = mlContext.Transforms.NormalizeLogMeanVariance("Features", useCdf: false);

In [ ]:
var normalizeTransform = normalize.Fit(data);
var transformedData = normalizeTransform.Transform(data);
var normalizeNoCdfTransform = normalizeNoCdf.Fit(data);
var noCdfData = normalizeNoCdfTransform.Transform(data);

In [ ]:
var column = transformedData.GetColumn<float[]>("Features").ToArray();
foreach (var row in column)
    Console.WriteLine(string.Join(", ", row.Select(x => x.ToString("f4"))));
var columnFixZero = noCdfData.GetColumn<float[]>("Features").ToArray();
foreach (var row in columnFixZero)
    Console.WriteLine(string.Join(", ", row.Select(x => x.ToString("f4"))));

In [ ]:
var transformParams = normalizeTransform.GetNormalizerModelParameters(0) as CdfNormalizerModelParameters<ImmutableArray<float>>;
Console.WriteLine("The 1-index value in resulting array would be produced by:");
Console.WriteLine("y = 0.5 * (1 + ERF((Math.Log(x) - " + transformParams.Mean[1] + ") / (" + transformParams.StandardDeviation[1] + " * Math.Sqrt(2))))");
var noCdfParams = normalizeNoCdfTransform.GetNormalizerModelParameters(0) as AffineNormalizerModelParameters<ImmutableArray<float>>;
var offset = noCdfParams.Offset.Length == 0 ? 0 : noCdfParams.Offset[1];
var scale = noCdfParams.Scale[1];
Console.WriteLine($"The 1-index value in resulting array would be produced by: y = (x - ({offset})) * {scale}");